# Loading the Netflix Clickstream Dataset into Pandas

The dataset is stored split up into many Parquet files in S3 (over 40,000 of them).
Parquet is a file format often used for storing large amounts of data efficiently.

Nonetheless, it will not be practical (or possible) to load all of the data onto your local machine and work with it in Pandas, as the following will demonstrate. 
To start gleaning insights from large datasets, data analysts usually choose a sample of the data to work with. This avoids having to downloading large amounts of files that may not fit onto a laptop's disk or into memory.

This notebook illustrates:

1. How to load a single Parquet file from S3 into Pandas using the `pd.read_parquet` function.
2. How to define a sample of files from S3 using `s3fs` and `pd.read_parquet`.
3. How to save the contents of DataFrames to your laptop to avoid having to load large datasets from the Cloud repeatedly.
4. How to restrict the set of columns to load into Pandas from S3.

The following assumes that you have `s3fs` and `pyarrow` installed.

## Exploring the dataset on S3

Files in S3 have addresses that look like `s3://s3-bucket-name/some/path/to/the/file`. The Netflix clickstream dataset is stored in under `s3://data-eng-datasets-404544469985/vod_clickstream/full/`.

We can list the files in the dataset by running the following command in a terminal:

```
aws s3 ls s3://data-eng-datasets-404544469985/vod_clickstream/full/ --recursive
```

This will list all of the files that make of up the dataset. The output will be very long.
Run the next cell to see an example (yes you can use `!` to run commands from Jupyter notebooks!):

In [ ]:
!aws s3 ls s3://data-eng-datasets-404544469985/vod_clickstream/full/ --recursive


But how do we get these files into Pandas? There are a couple of bits to it:

- How to read Parquet files using Pandas
- How to read data from S3
- How to get a subset of the files into Pandas to avoid running out of space and memory on your laptop

Read on to find out.

## Loading a single Parquet file from S3 into Pandas

We can use [`pd.read_parquet`](https://pandas.pydata.org/docs/reference/api/pandas.read_parquet.html) to load a Parquet file into a Pandas DataFrame.
With `s3fs` installed, it's enough to supply the S3 path to the file (and we can get paths to files using `aws s3 ls` as illustrated above). The file will be downloaded from S3 to your laptop, then loaded into a DataFrame.

Here's an example of using `pd.read_parquet`:

In [ ]:
import pandas as pd

netflix_sample_df = pd.read_parquet(
    f"s3://data-eng-datasets-404544469985/vod_clickstream/full/dt=2016-01-01/0000_part_00.parquet",
)
netflix_sample_df.head()


And we've got some data. The meaning of the columns will probably not be clear to you yet, but that's expected. 
You'll have to chance to dive into that later on.

For now, let's work out how we can load multiple files into Pandas at once.

# Selecting a subset of files to load into Pandas from S3

One approach to loading multiple files into a DataFrame would be to first figure out which files we want, then load them all into separate DataFrames as shown above, and finally to join all these DataFrames into one large one. But that's cumbersome!

Luckily, there are easier ways.
One of them is to use a [glob pattern](https://en.wikipedia.org/wiki/Glob_(programming)). 
Glob patterns allow us to specify sets of filenames with wildcard characters.

You can try this on your machine now. Navigate to a location where you have some Python files stored on your laptop, then list the files in that directory

```
cd path/to/some/python/project
ls
```

This will show all the files you have in your project.
But what if you only wanted to list the Python files? Run the following:

```
ls *.py
```

This will list only the files that end in `.py`.
Here, `*` is a wildcard standing for "any string of characters except /" and `*.py` is a glob pattern.

We're going to use glob patterns and the `s3fs` library to select a sample of the full dataset from S3.

### Using glob patterns to select multiple S3 files at once

The files of our dataset have S3 filepaths of the form:

```
s3://data-eng-datasets-404544469985/vod_clickstream/full/dt=2016-01-01/0000_part_00.parquet
```

Note that the filepaths includes the date as `dt=2016-01-01`, meaning that this particular file contains user activity from the 1st January 2016. Let's say we wanted to focus our analysis on the January 1st 2016 only.

To make a glob pattern that covers all of the Parquet files for January 1st, we'll have to replace the filename with `*`:

```
s3://data-eng-datasets-404544469985/vod_clickstream/full/dt=2016-01-01/*.parquet
```

And because we know all the files end in `.parquet`, we can simplify the above to omit that part and end up with this glob pattern:

```
s3://data-eng-datasets-404544469985/vod_clickstream/full/dt=2016-01-01/*
```

This will match all files that contain user activity from January 1st 2016.
Now, let's use `s3fs` to list out all of the matching files:

In [ ]:
# Get the list of files you are interested in.
import s3fs

s3 = s3fs.S3FileSystem(anon=False)
files = s3.glob('s3://data-eng-datasets-404544469985/vod_clickstream/full/dt=2016-01-01/*')
len(files)

Running the above cell should print the number 32, meaning that `s3fs` has found 32 files. We can inspect the filenames to see if they look correct.

> Note: this might take a few seconds to run.

In [ ]:
files

['data-eng-datasets-404544469985/vod_clickstream/full/dt=2016-01-01/0000_part_00.parquet',
 'data-eng-datasets-404544469985/vod_clickstream/full/dt=2016-01-01/0001_part_00.parquet',
 'data-eng-datasets-404544469985/vod_clickstream/full/dt=2016-01-01/0002_part_00.parquet',
 'data-eng-datasets-404544469985/vod_clickstream/full/dt=2016-01-01/0003_part_00.parquet',
 'data-eng-datasets-404544469985/vod_clickstream/full/dt=2016-01-01/0004_part_00.parquet',
 'data-eng-datasets-404544469985/vod_clickstream/full/dt=2016-01-01/0005_part_00.parquet',
 'data-eng-datasets-404544469985/vod_clickstream/full/dt=2016-01-01/0006_part_00.parquet',
 'data-eng-datasets-404544469985/vod_clickstream/full/dt=2016-01-01/0007_part_00.parquet',
 'data-eng-datasets-404544469985/vod_clickstream/full/dt=2016-01-01/0008_part_00.parquet',
 'data-eng-datasets-404544469985/vod_clickstream/full/dt=2016-01-01/0009_part_00.parquet',
 'data-eng-datasets-404544469985/vod_clickstream/full/dt=2016-01-01/0010_part_00.parquet',

You should see a list of file paths like this one:

```
data-eng-datasets-404544469985/vod_clickstream/full/dt=2016-01-01/0000_part_00.parquet
```

You might notice that the `s3://` prefix is missing. But that's okay. It just means that we have to remember to supply it when we pass these filenames to pandas.

Let's do that next. We'll read each of these files into a separate DataFrame using a for loop.



In [ ]:
# Read them into pandas + concat the dfs
dfs = []
for file in files:
    df = pd.read_parquet(f"s3://{file}")
    dfs.append(df)
len(dfs)

We now have data across 32 different DataFrames. We can concatenate them all into a single DataFrame using [pd.concat](https://pandas.pydata.org/docs/reference/api/pandas.concat.html).

In [ ]:
netflix_2016jan1_df = pd.concat(dfs)
netflix_2016jan1_df

Using s3fs and glob patterns, we've loaded a whole day of data into Pandas without having to manually write out the name of each file we wanted to load.

### Loading more than one day of data

On my machine, it took 6 seconds for all of these files to be loaded, which is longer than you're probably used to waiting for Pandas to load a dataset. But a day of data isn't even all that much. 

As an experiment, the next cell loads the whole month of January using the following glob pattern `s3://data-eng-datasets-404544469985/vod_clickstream/full/dt=2016-01-*/*`.

The code below is likely to take more than 5 minutes to run because it will take a while to download these files from S3.
Longer waiting times is often something we have to contend with when working with large datasets locally.
For this reason, let's also add some progress indicators so we have an idea of what's going on while we wait for it to load:

In [ ]:
january_2016_files = s3.glob('s3://data-eng-datasets-404544469985/vod_clickstream/full/dt=2016-01-*/*')
print(f"Found {len(january_2016_files)} files.")

dfs = []
for i, file in enumerate(january_2016_files):
    print(f"Loading file {i + 1}/{len(january_2016_files)} into a DataFrame ...")
    df = pd.read_parquet(f"s3://{file}")
    dfs.append(df)
len(dfs)



And let's concatenate the resulting DataFrames again:

In [ ]:
df_january_2016 = pd.concat(dfs)
df_january_2016

The resulting dataframe has more than 4.5 million rows. 

### Saving your work

To avoid having to download the data again (you might accidentally close your notebook at the end of a day or overwrite your variables), it's a good idea to save the contents of the DataFrame to your laptop locally so you can pick up from where you left off:


In [23]:
df_january_2016.to_parquet("netflix_jan_2016.parquet")

The above code saves the dataframe as a Parquet file named `netflix_jan_2016.parquet` in the current directory. This is especiallly useful if you've ended up making some transformations to the data in the DataFrame and you don't want to have to start from scratch. 

Next time you want to load the data, you can just use `pd.from_parquet("netflix_jan_2016.parquet")` to retrieve it without having to download it again from S3.

> Note: You don't have to save the data as a Parquet file. Pandas has functions for saving to [CSV](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html), Parquet and many other formats. But for large DataFrames, Parquet is probably the best choice.

### More glob patterns

We've seen how to select all files for a specific date and for a whole month:

```
jan1_files = s3.glob('s3://data-eng-datasets-404544469985/vod_clickstream/full/dt=2016-01-01/*')
all_jan_files = s3.glob('s3://data-eng-datasets-404544469985/vod_clickstream/full/dt=2016-01-*/*')
```

To express more complex conditions, you can use `[]` to express a range of acceptable characters. For example, this glob pattern matches the first 3 months of 2016:

```
2016q1_files = s3.glob('s3://data-eng-datasets-404544469985/vod_clickstream/full/dt=2016-0[1-3]-*/*')
```

`[...]` can also be used to list a set of characters to match. The following pattern matches January and May 2016: 

```
2016_jan_may_files = s3.glob('s3://data-eng-datasets-404544469985/vod_clickstream/full/dt=2016-0[15]-*/*')
```

Not everything will be easily expressible using a single glob pattern. 
You can always combine file lists generated by separate globs by concatenating them:

```
2016_jan_files = s3.glob('s3://data-eng-datasets-404544469985/vod_clickstream/full/dt=2016-01-*/*')
2017_dec_files = s3.glob('s3://data-eng-datasets-404544469985/vod_clickstream/full/dt=2017-12-*/*')

all_files = 2016_jan_files + 2017_dec_files
```

Note that all of these may take quite a significant time to load. It's best to get started with a small sample that does download in an acceptable time and see what you can glean from it.



### A convenient function for loading samples

For your convenience, below is a function that encapsulates the code that loads files using a glob pattern into a DataFrame and prints out some progress indicators. Feel free to use it in your own analysis and adapt it for your purposes.

In [ ]:
import s3fs
import pandas as pd

def load_parquet_files_from_s3(glob_pattern):
    s3 = s3fs.S3FileSystem(anon=False)
    files = s3.glob(glob_pattern)
    print(f"Found {len(files)} files.")

    dfs = []
    for i, file in enumerate(files):
        print(f"Loading file {i + 1}/{len(files)} into a DataFrame ...")
        df = pd.read_parquet(f"s3://{file}")
        dfs.append(df)
    return pd.concat(dfs)

df_jan1_2016 = load_parquet_files_from_s3("s3://data-eng-datasets-404544469985/vod_clickstream/full/dt=2016-01-01/*")

In [ ]:
df_jan1_2016.head()

## Restricting data size by filtering out columns we don't need

Another challenge of working with large datasets is that large Pandas DataFrames can become unwiedly. Holding on to a large Pandas DataFrame can make your computer slower. For example, running Pandas operations like `.groupby` and other common functions can take a while.

This is because large DataFrames take up a lot of space in your computer's working memory, which leaves less space for storing temporary results of any other operations you're asking the computer to do. Those temporary results then have to be stored in slower forms of storage like SSD or hard disk.

Running `.info()` on a DataFrame can give you a sense of this. It prints out a bunch of information including how much memory the DataFrame is taking up in the last line of the output:





In [ ]:
df_january_2016.info()

At the bottom of the output, you'll see a line like `memory usage: 2.0+ GB`. 
The DataFrame for January occupies more than 2 GB in memory. One way to reduce memory usage is to avoid loading columns we don't need.
Looking at the DataFrame we already loaded, we can see that the dataset has a lot of columns:


In [ ]:
len(df_january_2016.columns)

In [ ]:
df_january_2016.columns

Do we need all of these? Probably not.
You might want to refine this selection some more when you start exploring the data by yourself, but for now, let's restrict to the ones that look the most interesting:

- The time and date of the user action (`datetime`)
- The URL the user visited (`event_url`)
- The user ID (`panelist_id`)
- The user's country (`server_request_country_code`)

Let's create a dataframe that only has those 4 columns and compare its memory usage to that of the original dataframe:

In [ ]:
df_january_2016_smaller = df_january_2016[["datetime", "event_url", "panelist_id", "server_request_country_code"]]
df_january_2016_smaller.info()

The 4-column dataframe uses just under 200 MB of memory. That's a big saving! 
We can clean up the bigger version of the dataset to release the memory using Python's `del` keyword:

In [32]:
# del df_january_2016  # Uncomment this line to delete the df_january_2016 Dataframe from memory. The df_january_2016 variable will be deleted.


Could we somehow avoid loading the all columns to begin with? As it turns out, yes. One of the features that makes Parquet files efficient is that you can select which columns of the data you want to read without having to load all of the columns into memory first. (This is one factor makes them better than CSV files for large datasets.)

That means that we can use `pandas.read_parquet` to only load certain columns by passing it a `columns` argument. Let's try it for a single Parquet file:

In [ ]:
df_one_file = pd.read_parquet(
    f"s3://data-eng-datasets-404544469985/vod_clickstream/full/dt=2016-01-01/0000_part_00.parquet",
    columns=["datetime", "event_url", "panelist_id", "server_request_country_code"]
)
df_one_file.columns

To allow you to easily specify the columns you want when loading more than one file, below is a new version of the `load_parquet_files_from_s3` function we defined above with an added `columns` parameter. Do use and adapt it in your own code.

In [ ]:
import s3fs
import pandas as pd

def load_parquet_files_from_s3(glob_pattern, columns):
    s3 = s3fs.S3FileSystem(anon=False)
    files = s3.glob(glob_pattern)
    print(f"Found {len(files)} files.")

    dfs = []
    for i, file in enumerate(files):
        print(f"Loading file {i + 1}/{len(files)} into a DataFrame ...")
        df = pd.read_parquet(f"s3://{file}", columns=columns)
        dfs.append(df)
    return pd.concat(dfs)

test_df = load_parquet_files_from_s3("s3://data-eng-datasets-404544469985/vod_clickstream/full/dt=2016-01-01/*.parquet", columns=["event_url"])
test_df.columns

## Wrapping up

You now have a few tools and tricks you can use to start exploring the clickstream dataset.

Another key thing to take away from this is: **the amount of data a data analyst can efficiently analyse on their own machine is limited**. This is why in many organisations, data analysts are supported by data engineers, who set up the systems and preprocessing of data that makes analysis possible.

For now though, don't worry if the sample you are able to work with is small. At this stage, you are just exploring what this dataset has to offer.

## Further Resources

- [Pandas documentation: tips for working with large datasets in Pandas.](https://pandas.pydata.org/docs/user_guide/scale.html)
- [tqdm - an easy to use progress bar library for Python](https://github.com/tqdm/tqdm). See [this section of their docs](https://github.com/tqdm/tqdm#ipython-jupyter-integration) for how to use it in Jupyter notebooks.